In [2]:
import pandas as pd
import json
from pathlib import Path
# Model Algorithms
from sklearn.manifold import TSNE
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import numpy as np

In [3]:
recipes = []
ingredients = []
ids = []
path = Path('~/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/')
for i, po in enumerate(path.expanduser().glob('*.json')):
    with po.open() as fh:
        try:
            r = json.load(fh)
        except Exception:
            print(str(po))
            continue
        recipes.append(r)
        ing = r.get('Ingredients')
        id = r.get('RecipeID')
        ids.append(id)
        if ing is not None:
            ingredients.append([i['Name'].lower() for i in ing if i is not None and i['Name'] is not None])
       


/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-131-68-87332.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-131-71-87778.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-181-96-1158777.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-164-249-1283792.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-205-249-1797822.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-55-19-156227.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-127-74-2431270.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-112-159-862538.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-55-62-166860.json
/Users/bridgitboulahanis/

In [4]:

ing = pd.DataFrame(ingredients, index=ids)
ing
print(len(ingredients))
print(len([i for i in ingredients if None in i]))

ingstrings = [' '.join(i) for i in ingredients]
ingstrings

51473
0


['garam masala fresh coriander leaves; fresh ginger root turmeric water ground coriander water ground cumin red pepper; hot rice tea leaves chana dal salt ghee cumin seed onion',
 'coarse polenta/cornmeal fl oz milk half stick butter gran luchito smoked chilli paste pint water salt parmesan ear of sweetcorn (or use a small tin) chorizo sausage king prawns (raw) scallions handful fresh coriander eggs lime olive oil salt pepper',
 'pasta shrimp butter fresh italian parsley red pepper flakes ground black pepper  kosher salt dry white wine garlic parmesan cheese grated lemon',
 'black pepper egg for hush puppies whole grain onion salt salt for the fish buttermilk baking soda baking powder corn flour cayenne fish',
 'shrimp peeled and deveined habanero pepper jicama red bell pepper green bell pepper yellow bell pepper purple onion tomato lime chopped cilantro tsp. sea salt or',
 'chicken tenderloins or 4 chicken breast filets  sweet onion honeu barbeque sauce  sugar  powdered garlic  black 

In [8]:
## add block here with stem/lem and dedup party
ingstrings

['garam masala fresh coriander leaves; fresh ginger root turmeric water ground coriander water ground cumin red pepper; hot rice tea leaves chana dal salt ghee cumin seed onion',
 'coarse polenta/cornmeal fl oz milk half stick butter gran luchito smoked chilli paste pint water salt parmesan ear of sweetcorn (or use a small tin) chorizo sausage king prawns (raw) scallions handful fresh coriander eggs lime olive oil salt pepper',
 'pasta shrimp butter fresh italian parsley red pepper flakes ground black pepper  kosher salt dry white wine garlic parmesan cheese grated lemon',
 'black pepper egg for hush puppies whole grain onion salt salt for the fish buttermilk baking soda baking powder corn flour cayenne fish',
 'shrimp peeled and deveined habanero pepper jicama red bell pepper green bell pepper yellow bell pepper purple onion tomato lime chopped cilantro tsp. sea salt or',
 'chicken tenderloins or 4 chicken breast filets  sweet onion honeu barbeque sauce  sugar  powdered garlic  black 

In [6]:
vect = TfidfVectorizer(min_df=1, stop_words="english")                                                                                                                                                                                                   
vect.fit(ingstrings)  
features = vect.transform(ingstrings)

cosine_similarities = linear_kernel(features[3:4], features).flatten()
related_docs_indices = cosine_similarities.argsort()[:-5:-1]
print(related_docs_indices)
print(cosine_similarities[related_docs_indices])
print("=" * 5)
for i in related_docs_indices: print(ingstrings[i])

[    3 37811 21973  4079]
[1.         0.54275713 0.53693193 0.49457185]
=====
black pepper egg for hush puppies whole grain onion salt salt for the fish buttermilk baking soda baking powder corn flour cayenne fish
sugar baking powder salt flour baking soda egg buttermilk cardamom
milk vegetable oil salt baking powder black pepper paprika hush puppies onion catfish fillets cornmeal cornmeal white pepper salt pepper milk egg
flour cornstarch baking soda baking powder salt water


In [6]:

pickle.dump(vect,open("bigoven/vect.pkl","wb"))
pickle.dump(features,open("bigoven/features.pkl","wb"))



In [7]:
#proof of concept
a = pickle.load(open("bigoven/vect.pkl", 'rb'))
b = pickle.load(open("bigoven/features.pkl", 'rb'))
c =['2 skinless and boneless chicken breasts, butterflied and then cut in half Sea salt and freshly ground black pepper All-purpose flour, for dredging 6 tablespoons unsalted butter 5 tablespoons extra-virgin olive oil 1/3 cup fresh lemon juice 1/2 cup chicken stock 1/4 cup brined capers, rinsed 1/3 cup fresh parsley, chopped']

nf = a.transform(c)

cs = linear_kernel(nf, b).flatten()
rdi = cs.argsort()[:-5:-1]
print(rdi)
print(cs[rdi])
print("=" * 5)
for i in rdi: print(ingstrings[i])




[  641 23815  4049 22664]
[0.60407817 0.50858338 0.48633755 0.48289028]
=====
1/2 cup
boneless, skinless chicken breasts salt and pepper all purpose flour unsalted butter extra-virgin olive oil fresh lemon juice chicken stock brined capers fresh parsley
ingredients  1 1/4 cup red wine vinegar  1/4 cup s
chicken breasts flour extra virgin olive oil unsalted butter fresh lemon juice chicken stock brined capers fresh parsley salt black pepper


In [8]:
ingredients[1]

['pasta',
 'shrimp',
 'butter',
 'fresh italian parsley',
 'red pepper flakes',
 'ground black pepper ',
 'kosher salt',
 'dry white wine',
 'garlic',
 'parmesan cheese grated',
 'lemon']

In [45]:
fm = pd.read_excel('greenmarketproducts.xlsx', index_col=0)
fmp = fm['TYPES OF PRODUCTS AVAILABLE'].tolist()
veggies = open('juneveggies.txt').read().strip().split(',')

In [46]:
allprod = fmp+veggies
ap = [i.split(',') for i in allprod]

cleanap = []
for line in ap:
    for i in line:
        cleanap.append(i.strip().lower())
        


In [47]:
fm

,TYPES OF PRODUCTS AVAILABLE,aisles
BUSINESS NAME,,
1857 Spirits,Potato Vodka,Alcoholic Beverages
Abundance Acres Farm,"chicken, duck, turkey, beef, pork, eggs, mapl...",meat
Alewife Farm,Vegetables,produce
Andrew's Local Honey,Honey,"Nut butters, jams, and honey"
Anthony Road Wine Co.,Wine,Alcoholic Beverages
...,...,...
Wilklow Orchards,"Vegetables, Fruits, Cider, Jams, Beef, Pork",produce
Willow Wisp Organic Farm,Vegetables,"nut butters, Jams, and Honey"
Wood Homestead,"Maple Syrup, Honey, Sorghum Syrup",produce


In [48]:
cleanap
pickle.dump(cleanap,open("bigoven/FMproducts.pkl","wb"))


In [12]:
perc = []
for num, i in enumerate(ingredients):
    if len(i) > 0:
        both = set(i).intersection(cleanap)
        perc.append(len(both)/len(i)*100)
    else:
        perc.append(0)
    
    

In [30]:
rec = pd.DataFrame(recipes)
rec['percent'] = perc
pickle.dump(rec,open("bigoven/recipes.pkl","wb"))
pickle.dump(cleanap,open("bigoven/FMproducts.pkl","wb"))


In [26]:
for i in [22664, 23351, 23815, 30977]:
    print(ingredients[i])
    print(perc[i])
    print('===================')

#ingredients[22664]#, 23351, 23815, 30977]

['chicken breasts', 'flour', 'extra virgin olive oil', 'unsalted butter', 'fresh lemon juice', 'chicken stock', 'brined capers', 'fresh parsley', 'salt', 'black pepper']
10.0
['chicken breasts', 'flour', 'parmesan cheese', 'salt', 'sweet butter', 'olive oil', 'lemon juice', 'chicken broth', 'brined capers', 'parsley']
40.0
['boneless, skinless chicken breasts', 'salt and pepper', 'all purpose flour', 'unsalted butter', 'extra-virgin olive oil', 'fresh lemon juice', 'chicken stock', 'brined capers', 'fresh parsley']
0.0
['bread crumbs', 'chicken mince', 'eggs', 'zucchini', 'carrot', 'onion', 'coriander leaves', 'flat-leaf parsley leaves', 'frozen puff pastry', 'sesame seeds', 'tomato or sweet chilli or sauce, to serve']
9.090909090909092
